In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import os
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import Resize
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset
from torch import Tensor
!pip install vit-pytorch

from vit_pytorch import ViT

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from numpy import load


from google.colab import drive
drive.mount('/content/drive')
x_train = load('/content/drive/MyDrive/Colab_data/x_train.npy')
y_train = load('/content/drive/MyDrive/Colab_data/y_train.npy')
x_test = load('/content/drive/MyDrive/Colab_data/x_test.npy')
y_test = load('/content/drive/MyDrive/Colab_data/y_test.npy')

# x_train = load('x_train.npy')
# y_train = load('y_train.npy')
# x_test = load('x_test.npy')
# y_test = load('y_test.npy')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
x_train = x_train.reshape((-1,3, 224, 224))
x_test = x_test.reshape((-1,3, 224, 224))
print(np.shape(x_train))

(3601, 3, 224, 224)


In [4]:
torch.cuda.is_available()

True

In [5]:
dataset_train = TensorDataset( Tensor(x_train), Tensor(y_train).long() )
dataset_test = TensorDataset( Tensor(x_test), Tensor(y_test).long())


image_size = 224

# Define transformations
train_transform = transforms.Compose([
    # transforms.RandomResizedCrop(image_size),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    # transforms.Resize(image_size),
    # transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformations to custom datasets
dataset_train = train_transform(x_train)
dataset_test = test_transform(x_test)

# Create DataLoader objects
batch_size = 128
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)




In [ ]:
# Initialize model, loss, and optimizer
num_epochs = 150
train_loss_rec = np.zeros(num_epochs)
train_err_rec = np.zeros(num_epochs)
val_err = np.zeros(num_epochs)
model = ViT(
    image_size = 224,
    patch_size = 32,
    num_classes = 2,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RAdam(model.parameters(), lr=1e-3, weight_decay=2e-2)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # Evaluation
    model.eval()
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()


        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    train_loss_rec[epoch] = 
    print(f"train_Accuracy: {100 * train_correct / train_total}%")
    print(f"test_Accuracy: {100 * test_correct / test_total}%")

Epoch 1/150, Loss: 0.727597791573097
train_Accuracy: 54.81810608164399%
test_Accuracy: 53.82907880133185%
Epoch 2/150, Loss: 0.7009080710082218
train_Accuracy: 57.067481255206886%
test_Accuracy: 53.16315205327414%
Epoch 3/150, Loss: 0.6938211732897265
train_Accuracy: 60.26103860038878%
test_Accuracy: 59.822419533851274%
Epoch 4/150, Loss: 0.7046447149638472
train_Accuracy: 58.84476534296029%
test_Accuracy: 57.49167591564928%
Epoch 5/150, Loss: 0.6577728287927036
train_Accuracy: 65.25965009719522%
test_Accuracy: 63.152053274139845%
Epoch 6/150, Loss: 0.6211882295279667
train_Accuracy: 70.53596223271313%
test_Accuracy: 69.70033296337402%
Epoch 7/150, Loss: 0.5893952640993841
train_Accuracy: 71.36906414884754%
test_Accuracy: 69.58934517203107%
Epoch 8/150, Loss: 0.6041547224439424
train_Accuracy: 57.03971119133574%
test_Accuracy: 53.9400665926748%
Epoch 9/150, Loss: 0.6218685980500847
train_Accuracy: 72.70202721466259%
test_Accuracy: 67.59156492785793%
Epoch 10/150, Loss: 0.57144129687342